In [1]:
import pandas as pd
from utils import get_market_df, plot_df_chart, PLOTS_FOLDER_PATH, DISAGGREGATED_FUT, print_json
from analysis import get_market_net, get_market_cotIndex, get_market_spreading, get_market_oi, get_market_participants

In [2]:
N = 261

MARKET = "SILVER"
REPORT_TYPE = DISAGGREGATED_FUT

In [3]:
df = get_market_df(cot_report_type=REPORT_TYPE, market="SILVER")

loading data for the report type: disaggregated_fut ...
Selected: Disaggregated Report Futures-only - loading historical data ...
Selected: Disaggregated Report Futures-only from year: 2017 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2018 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2019 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2020 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2021 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2022 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2023 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2024 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2025 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2026 - loading data ...


- **pick participants**

In [4]:
participants = get_market_participants(df)

list(participants.keys())

['prod_merc', 'swap', 'm_money', 'other_rept', 'tot_rept', 'nonrept']

In [5]:
participants = {k: v for k, v in participants.items() if k in ["m_money", "prod_merc"]}

print_json(participants)

{
        "prod_merc": {
                "oi_all": "open_interest_all",
                "long": "prod_merc_positions_long_all",
                "short": "prod_merc_positions_short_all",
                "long_oi": "pct_of_oi_prod_merc_long_all",
                "short_oi": "pct_of_oi_prod_merc_short_all",
                "long_change": "change_in_prod_merc_long_all",
                "short_change": "change_in_prod_merc_short_all"
        },
        "m_money": {
                "oi_all": "open_interest_all",
                "long": "m_money_positions_long_all",
                "short": "m_money_positions_short_all",
                "spread": "m_money_positions_spread_all",
                "long_oi": "pct_of_oi_m_money_long_all",
                "short_oi": "pct_of_oi_m_money_short_all",
                "spread_oi": "pct_of_oi_m_money_spread_all",
                "long_change": "change_in_m_money_long_all",
                "short_change": "change_in_m_money_short_all",
                "sp

In [6]:
df_net = get_market_net(market_df=df, participants=participants)
df_index = get_market_cotIndex(market_df=df, participants=participants, window=N)
df_spreading = get_market_spreading(market_df=df, participants=participants)
df_oi = get_market_oi(market_df=df, participants=participants)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("Net positions", f"COT index ({N} weeks)", 'OI', 'Spreading'))

for col in df_net.columns:
    fig.add_trace(
        go.Bar(
            x=df_net.index,
            y=df_net[col],
            name=col,
            legendgroup=col,
        ),
        row=1, col=1
    )

for col in df_index.columns:
    fig.add_trace(
        go.Scatter(
            x=df_index.index,
            y=df_index[col],
            name=col,
            legendgroup=col
        ),
        row=2, col=1
    )

for col in df_oi.columns:
    fig.add_trace(
        go.Scatter(
            x=df_oi.index,
            y=df_oi[col],
            name=col,
            legendgroup=col
        ),
        row=3, col=1
    )

for col in df_spreading.columns:
    fig.add_trace(
        go.Scatter(
            x=df_spreading.index,
            y=df_spreading[col],
            name=col,
            legendgroup=col
        ),
        row=4, col=1
    )

fig.add_shape(
    type='rect',
    xref='paper',  # spans the full width of the plot
    yref='y2',  # y-axis of subplot in row 1
    x0=0,
    x1=1,
    y0=80,       # Adjust based on your y-axis range
    y1=100,
    fillcolor='green',
    opacity=0.2,
    line_width=0,
    layer='below'
)

fig.add_shape(
    type='rect',
    xref='paper',  # spans the full width of the plot
    yref='y2',  # y-axis of subplot in row 1
    x0=0,
    x1=1,
    y0=0,              # Adjust based on your y-axis range
    y1=20,
    fillcolor='red',
    opacity=0.2,
    line_width=0,
    layer='below'
)

# Update layout
fig.update_layout(
    height=2000,
    width=1450,
    title_text=f"COT Analysis for EURO FX",
    template='plotly_dark',
    hovermode='x unified',  # or 'x unified' or 'closest',
)

fig.update_yaxes(title_text='CONTRACT')

In [10]:
fig.write_html(f"{PLOTS_FOLDER_PATH}/analysis.html", auto_open=False, full_html=True, include_plotlyjs='cdn', config={'responsive': True})

print(f"[->] Plot saved as '{PLOTS_FOLDER_PATH}/analysis.html'.")

[->] Plot saved as '/home/mod7ex/COT-analysis/plots/analysis.html'.
